In [1]:
import pandas as pd
import pandasql as pdsql
import os
import datetime
from collections import deque, Counter

# Read and preprocess the input

In [2]:
datadir = '/home/AzureUser/data/order_brushing'
input_filename = 'order_brush_order.csv'
input_filepath = os.path.join(datadir, input_filename)

input_df = pd.read_csv(input_filepath)
input_df.head(5)

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [3]:
# Convert event_time to datetime format
def str2datetime(s):
    return datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S')

input_df['event_time'] = input_df['event_time'].apply(str2datetime)

# Get possible order brushing time range

From the problems, an order is considered as part of order brushing if there is an hour timerange that have concentration rate more than or equal to 3.
Since time is real number and continuous, there are infinitely many timerange that is possible for order brushing.
But our goal is just to see whether an order is a part of order brushing or not, hence if we have many timeranges with the same orders, we just need to take one of them!

One way to make the number of timerange finite, is to discretize. Since the smallest time unit in event_time is second, we can possibly consider all timerange with half second increment.
This discretization still got a lot of possible timerange, and we still got many timerange with the same orders!

To reduce the possible time-range further, we should consider the interaction between and order and possible time-range.
Suppose we have order at time t. The possible different interaction of time-range with the order are as follow:
1. Time-range that end just before time t  -> timerange.end = t - epsilon
2. Time-range that end exactly at time t   -> timerange.end = t
3. Time-range that start exactly at time t -> timerange.start = t
4. Time-range that start just after time t -> timerange.start = t + epsilon

And if we have a multiple orders and a timerange TR, we could always move timerange TR to either start or end nearby an order, hence satisfy one of the condition 1-4.

Since the timerange have fixed length of 1 hour. We can write the 4 conditions with timerange.start only:
1. timerange.start + 1 hour = t - epsilon -> timerange.start = t - 1 hour - epsilon
2. timerange.start + 1 hour = t           -> timerange.start = t - 1 hour
3. timerange.start = t                    -> timerange.start = t
4. timerange.start = t + epsilon          -> timerange.start = t + epsilon

furthermore since the smallest unit of possible t is 1 second, hence we can set epsilon as 0.5 s.

Notes: the timerange should be associated with its shopid


In [4]:
# Create a set of event_times for every shopid
# Mapping shopid to set of event_times
event_times_dict = {}

for shopid, event_time in input_df[['shopid', 'event_time']].to_numpy():
    if shopid not in event_times_dict:
        event_times_dict[shopid] = set()
    event_times_dict[shopid].add(event_time)


In [5]:
# Expand event_times for each shopid and sort
timeranges = {}

for shopid, event_times in event_times_dict.items():
    timerange_start1 = {t - datetime.timedelta(hours=1, seconds=0.5) for t in event_times}
    timerange_start2 = {t - datetime.timedelta(hours=1) for t in event_times}
    timerange_start3 = {t for t in event_times}
    timerange_start4 = {t + datetime.timedelta(seconds=0.5) for t in event_times}
    timeranges[shopid] = sorted(timerange_start1 | timerange_start2 | timerange_start3 | timerange_start4)

# Identify order brushing time range
For every timerange, we should define it as order brushing if the number of order divided by number of unique user (concentration rate) is more or equal to 3.

We should identify all the order brushing timerange, and save the order associated with it.

In [6]:
# Get sorted list of (orderid, userid, event_time) for every shopid

orders_usertime = {}
for shopid, orderid, userid, event_time in input_df[['shopid', 'orderid', 'userid', 'event_time']].to_numpy():
    if shopid not in orders_usertime:
        orders_usertime[shopid] = []
    orders_usertime[shopid].append((orderid, userid, event_time))

for shopid in orders_usertime:
    orders_usertime[shopid].sort(key=lambda x: (x[2], x[0]))

In [7]:
# On every shopid, we move the timerange windows associated with it. 
# We will have a queue for the windows, push the orders that still within its range and pop the orders that already outside its range.
# On every window, we will check if it is order_brushing, and save it if it is indeed order_brushing
# The overall complexity should be O(#order + #timerange)

# Save 
order_brushing = {}

for shopid in timeranges:
    cur_timeranges = timeranges[shopid]
    cur_orders = orders_usertime[shopid]
    q = deque()
    last_q_index = 0 #Save the last added q_index to order_brushing
    last_order_index = 0
    # We keep track of user_count, num_order, and num_unique_user to optimize the computation
    user_count = Counter()
    num_order = 0
    num_unique_user = 0
    for timerange_start in cur_timeranges:
        timerange_end = timerange_start + datetime.timedelta(hours=1)
        # Add new order into q
        while last_order_index < len(cur_orders) and cur_orders[last_order_index][2] <= timerange_end:
            orderid, userid, _ = cur_orders[last_order_index]
            q.append(cur_orders[last_order_index])
            num_order += 1
            if user_count[userid] == 0:
                num_unique_user += 1
            user_count[userid] += 1
            last_order_index += 1
        # Remove obsolete order from q
        while len(q) > 0 and q[0][2] < timerange_start:
            orderid, userid, _ = q.popleft()
            num_order -= 1
            user_count[userid] -= 1
            if user_count[userid] == 0:
                user_count.pop(userid, None)
                num_unique_user -= 1
            # Adjust offset
            if last_q_index > 0:
                last_q_index -= 1
            
        # Analyze the current q content
        if num_order > 0 and num_order >= num_unique_user * 3:
            # Concentration rate >= 3 --> order brushing!
            if shopid not in order_brushing:
                order_brushing[shopid] = []
                # print("Add order_brushing for shopid: {}, current q: {}".format(shopid, q))
            for i in range(last_q_index, len(q)):
                orderid, userid, event_time = q[i]
                order_brushing[shopid].append((orderid, userid, event_time))
            # Update last_q_index so we dont add duplicate data into order_brushing
            last_q_index = len(q)
    

In [8]:
len(order_brushing)

315

# Identify the most suspicious users for every shopid
The most suspicious users is defined as the users that have most order that is part of order brushing in a particular shop

## For every shopid, lets count the number of orders per user

In [9]:
user_order_count = {}

for shopid in order_brushing:
    if shopid not in user_order_count:
        user_order_count[shopid] = Counter()
    for orderid, userid, event_time in order_brushing[shopid]:
        user_order_count[shopid][userid] += 1

## Get the maximum number of order per user in each shopid

In [10]:
user_order_max = {}
for shopid in user_order_count:
    user_order_max[shopid] = user_order_count[shopid].most_common(1)[0][1]

## Filter the user, just take the user with maximum number of order per shopid

In [11]:
filtered_user = {}
for shopid in user_order_count:
    filtered_user[shopid] = sorted([user for user, count in user_order_count[shopid].items() if count == user_order_max[shopid]])

# Now, lets wrap up and build the result csv

## First, lets get the unique shopid

In [12]:
shopids = sorted(set(input_df['shopid']))

## Build the suspicious user as same format as required

In [13]:
def safe_convert(users):
    if len(users) == 0:
        return '0'
    else:
        return '&'.join([str(user) for user in users])

suspicious_users = [safe_convert(filtered_user.get(shopid, [])) for shopid in shopids ]

## Now lets build the dataframe containing shopid and corresponding suspicious user

In [14]:
result_df = pd.DataFrame({
    'shopid': pd.Series(shopids),
    'userid': pd.Series(suspicious_users)
})

## Save into csv

In [15]:
result_df.to_csv('order_brushing_result.csv', index=False)